In [1]:
import pandas as pd
import os, pdb, sys
from tqdm import tqdm

In [2]:
sys.getrecursionlimit()

3000

In [3]:
sys.setrecursionlimit(100)

In [4]:
langs = [d for d in os.listdir(os.getcwd()) if os.path.isdir(d) and not d.startswith('.')]
len(langs)

15

In [5]:
def load_data(lang):
    deriv = pd.read_csv(f'{lang}/{lang}.derivational.v1.tsv',sep='\t',names=['source_word','target_word','source_pos','target_pos','morpheme','type'])
    infl = pd.read_csv(f'{lang}/{lang}.inflectional.v1.tsv',sep='\t',names=['lemma','inflected_form','morph_feat','morph_seg'])
    return deriv,infl

In [21]:
deriv,infl = load_data('deu')

In [23]:
deriv.source_word.fillna('null',inplace=True)

In [24]:
deriv

,source_word,target_word,source_pos,target_pos,morpheme,type
0,axial,abaxial,J,J,ab,prefix
1,Laut,ablaut,N,N,ab,prefix
2,China,Chinese,N,N,ese,suffix
3,Liste,listen,N,V,en,suffix
4,fallen,befallen,V,V,be,prefix
...,...,...,...,...,...,...
29376,finden,hereinfinden,V,V,herein,prefix
29377,finden,hinfinden,V,V,hin,prefix
29378,Bulgarien,Bulgarier,N,N,er,suffix
29379,Streit,Streiterei,N,N,erei,suffix


In [25]:
deriv[deriv.target_word.str.contains('agriculture')]

,source_word,target_word,source_pos,target_pos,morpheme,type


In [26]:
def get_segment(source_word,segmentation=None,typ=None):
    # pdb.set_trace()
    result = deriv.query(f"target_word=='{source_word}'")
    if len(result)==0:
        return (source_word,)
    else:
        source_word = result.source_word.values[0]
        morpheme = result.morpheme.values[0]
        typ = result.type.values[0]
        
        previous_segment = get_segment(source_word)
        
        if typ=='prefix':
            return morpheme+'#',*previous_segment
        elif typ=='suffix':
            return *previous_segment,'#'+morpheme

In [9]:
s = get_segment('agricultural')
s

('agriculture', '#al')

In [27]:
segments={}
for row in tqdm(deriv.iterrows(),total=len(deriv)):
    index = row[0]
    try:
        seg = get_segment(row[1].target_word)
        segments[index]=seg
    except Exception as e:
        segments[index]=e

100%|██████████| 29381/29381 [01:19<00:00, 369.24it/s]


In [28]:
deriv['segments']=pd.Series(segments)

In [29]:
deriv

,source_word,target_word,source_pos,target_pos,morpheme,type,segments
0,axial,abaxial,J,J,ab,prefix,"(ab#, axial)"
1,Laut,ablaut,N,N,ab,prefix,"(ab#, Laut)"
2,China,Chinese,N,N,ese,suffix,"(China, #ese)"
3,Liste,listen,N,V,en,suffix,"(Liste, #en)"
4,fallen,befallen,V,V,be,prefix,"(be#, fallen)"
...,...,...,...,...,...,...,...
29376,finden,hereinfinden,V,V,herein,prefix,"(herein#, finden)"
29377,finden,hinfinden,V,V,hin,prefix,"(hin#, finden)"
29378,Bulgarien,Bulgarier,N,N,er,suffix,"(Bulgarien, #er)"
29379,Streit,Streiterei,N,N,erei,suffix,"(Streit, #erei)"


In [31]:
deriv.query(f"target_word=='finden'")

,source_word,target_word,source_pos,target_pos,morpheme,type,segments


In [30]:
infl

,lemma,inflected_form,morph_feat,morph_seg
0,zufügen,zufügen,V;NFIN,-
1,zufügen,zufügend,V;V.PTCP;PRS,zu-|füg|-end
2,zufügen,zugefügt,V;V.PTCP;PST,zu-|ge-|füg|-t
3,zufügen,füge zu,V|IND;PRS;1;SG ADP,füg|-e zu
4,zufügen,fügen zu,V|IND;PRS;1;PL ADP,füg|-en zu
...,...,...,...,...
490326,Schifffahrt,Schifffahrten,N|ACC;PL,Schifffahrt|en
490327,Katzenstreu,Katzenstreu,N;NOM;SG;FEM,-
490328,Katzenstreu,Katzenstreu,N;GEN;SG;FEM,-
490329,Katzenstreu,Katzenstreu,N;DAT;SG;FEM,-


In [20]:
infl.morph_seg.apply(lambda e: e.split('|'))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/jabbar/miniconda3/envs/nlp/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_952526/1434261198.py", line 1, in <module>
    infl.morph_seg.apply(lambda e: e.split('|'))
  File "/home/jabbar/miniconda3/envs/nlp/lib/python3.10/site-packages/pandas/core/series.py", line 4771, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "/home/jabbar/miniconda3/envs/nlp/lib/python3.10/site-packages/pandas/core/apply.py", line 1105, in apply
    return self.apply_standard()
  File "/home/jabbar/miniconda3/envs/nlp/lib/python3.10/site-packages/pandas/core/apply.py", line 1156, in apply_standard
    mapped = lib.map_infer(
  File "pandas/_libs/lib.pyx", line 2918, in pandas._libs.lib.map_infer
  File "/tmp/ipykernel_952526/1434261198.py", line 1, in <lambda>
    infl.morph_seg.apply(lambda e: e.split('|'))